# Naive Bayesian Classifier
### Q1. Bayes Rule을 이해하고 Naive  Bayes classifier가 사용하는 사후 확률 계산 과정을 서술하세요.

- Bayes Rule:   
$P(w_i|x) = \frac{P(x|w_i)|P(w_i)}{P(x)} = \frac{P(x|w_i) P(w_i)}{\Sigma_j P(x|w_j)P(w_j)}$
  -
  - $P(x|w_i)\text{: 사후 확률, posterior}\\
P(x|w_i) \text{: 가능도/우도, likelihood}\\
P(w_i) \text{: 사전 확률, prior}\\
P(x) \text{: 증거, evidence}$

A1. 각 클래스에 대한 사전 확률을 계산하고 각 클래스 wi에 대해 주어진 데이터가 나타날 확률을 계산한다. 나이브 베이즈에서는 모든 특징들이 서로 독립이라고 가정하므로 우도는 각 특징의 조건부 확률의 곱으로 표현된다. 모든 클래스에 대해 계산된 우도와 사전 확률의 곱을 모두 더한 값으로 증거를 계산한다. 마지막으로 우도와 사전 확률의 곱을 증거로 나눈 값으로 사후 확률을 계산한다.

### Q2. Naive Bayes Classification 방법을 이용해서 다음 생성된 리뷰 데이터에 기반한 감정 분석을 해봅시다.

In [34]:
# pip install pandas
import pandas as pd
import re

In [35]:
# 리뷰 데이터 생성
data = {
    'review': [
        'I love this great product! It exceeded my expectations.',
        'The Worst purchase I have ever made. Completely useless.',
        'It is an average product, nothing special but not terrible either.',
        'Great service and who can help but love this design? Highly recommend!',
        'Terrible experience, I will never buy from this poor brand again.',
        'It’s acceptable, but I expected better service, not just an acceptable one.',
        'Absolutely wonderful! I am very satisfied with this great service.',
        'The quality is poor and it broke after one use. Terrible enough!',
        'Acceptable product for the price, but there are better options out there.',
        'Great quality and fast shipping with wonderful service! I love it'
    ],
    'sentiment': [
        'positive', 'negative', 'neutral', 'positive', 'negative',
        'neutral', 'positive', 'negative', 'neutral', 'positive',
    ]
}
df = pd.DataFrame(data)
df.head()

,review,sentiment
0,I love this great product! It exceeded my expe...,positive
1,The Worst purchase I have ever made. Completel...,negative
2,"It is an average product, nothing special but ...",neutral
3,Great service and who can help but love this d...,positive
4,"Terrible experience, I will never buy from thi...",negative


In [36]:
# 불용어 리스트 정의
stopwords = ['i', 'my', 'am', 'this', 'it', 'its', 'an', 'a', 'the', 'is', 'are', 'and', 'product', 'service']

In [37]:
# 텍스트 전처리 함수 정의
def preprocess_text(text):
    # 소문자로 변환
    text = text.lower()
    # 특수 기호 제거
    text = re.sub(r'[^a-z\s]', '', text)
    # 불용어 제거
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords]
    return ' '.join(filtered_words)

# 모든 리뷰에 대해 전처리 수행
df['review'] = df['review'].apply(preprocess_text)

기본적인 데이터 전처리가 완료되었습니다!
이제부터 직접 나이브 베이지안 분류를 수행해 봅시다.  
우리가 분류하고자 하는 문장은 총 두가지 입니다.  
전처리가 완료되었다고 치고,   
첫번째 문장은 **'love, great, awesome'**,  
두번째 문장은 **'terrible, not, never'** 입니다.

사전 확률 $P(positive), P(negative), P(neutral)$을 구합니다.

In [69]:
# 각 클래스의 빈도를 계산
class_counts = df['sentiment'].value_counts()

# 전체 리뷰 수
total_reviews = len(df)

# 사전 확률 계산
prior_probabilities = class_counts / total_reviews

# 사전 확률 출력
print(prior_probabilities)





sentiment
positive    0.4
negative    0.3
neutral     0.3
Name: count, dtype: float64


가능도를 구하기 위한 확률들을 계산합니다.  
예: 첫번째 문장 분류를 위해서는, $P(love|positive), P(great|positive), P(awesome|positive)\\
P(love|negative), P(great|negative), P(awesome|negative)\\
P(love|neutral), P(great|neutral), P(great|neutral)$를 구합니다.

이 때 CountVectorizer를 사용하여 도출한 단어 벡터를 활용하면 확률들을 간편하게 구할 수 있습니다.  
참고: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [70]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
review_array = vectorizer.fit_transform(df['review']).toarray()
review_array

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 2, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 

In [68]:
vectorizer.get_feature_names_out()

array(['love great awesome'], dtype=object)

In [41]:
vectorizer.vocabulary_

{'love': 28,
 'great': 23,
 'exceeded': 16,
 'expectations': 17,
 'worst': 53,
 'purchase': 38,
 'have': 24,
 'ever': 15,
 'made': 29,
 'completely': 11,
 'useless': 47,
 'average': 4,
 'nothing': 32,
 'special': 43,
 'but': 8,
 'not': 31,
 'terrible': 44,
 'either': 13,
 'who': 49,
 'can': 10,
 'help': 25,
 'design': 12,
 'highly': 26,
 'recommend': 40,
 'experience': 19,
 'will': 50,
 'never': 30,
 'buy': 9,
 'from': 22,
 'poor': 36,
 'brand': 6,
 'again': 3,
 'acceptable': 1,
 'expected': 18,
 'better': 5,
 'just': 27,
 'one': 33,
 'absolutely': 0,
 'wonderful': 52,
 'very': 48,
 'satisfied': 41,
 'with': 51,
 'quality': 39,
 'broke': 7,
 'after': 2,
 'use': 46,
 'enough': 14,
 'for': 21,
 'price': 37,
 'there': 45,
 'options': 34,
 'out': 35,
 'fast': 20,
 'shipping': 42}

In [42]:
frequency_matrix = pd.DataFrame(review_array, columns = vectorizer.get_feature_names_out())
frequency_matrix = pd.concat([df['sentiment'], frequency_matrix], axis=1)
frequency_matrix

,sentiment,absolutely,acceptable,after,again,average,better,brand,broke,but,...,terrible,there,use,useless,very,who,will,with,wonderful,worst
0,positive,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,negative,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,neutral,0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
3,positive,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,negative,0,0,0,1,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,0
5,neutral,0,2,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
6,positive,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,1,0
7,negative,0,0,1,0,0,0,0,1,0,...,1,0,1,0,0,0,0,0,0,0
8,neutral,0,1,0,0,0,1,0,0,1,...,0,2,0,0,0,0,0,0,0,0
9,positive,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0


In [71]:
# 위와 같이 조건부 확률을 구하는 코드를 작성해주세요

from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np

# Vectorizer로 변환
vectorizer = CountVectorizer()
review_array = vectorizer.fit_transform(df['review']).toarray()

# 단어 리스트 가져오기
words = vectorizer.get_feature_names_out()

# 단어의 빈도 매트릭스 생성
frequency_matrix = pd.DataFrame(review_array, columns=words)

# 원본 데이터프레임과 단어 빈도 매트릭스 결합
frequency_matrix = pd.concat([df['sentiment'], frequency_matrix], axis=1)

# 클래스별 단어 빈도 합산
class_word_frequencies = frequency_matrix.groupby('sentiment').sum()

# 클래스별 단어 빈도 합계 계산
class_word_totals = class_word_frequencies.sum(axis=1)

# 스무딩 파라미터 설정
alpha = 1

# 조건부 확률 계산
conditional_probabilities = (class_word_frequencies + alpha) / (class_word_totals.to_numpy()[:, np.newaxis] + alpha * len(words))

# 조건부 확률 출력
print(conditional_probabilities)






           absolutely  acceptable     after     again   average    better  \
sentiment                                                                   
negative     0.012821    0.012821  0.025641  0.025641  0.012821  0.012821   
neutral      0.012821    0.051282  0.012821  0.012821  0.025641  0.038462   
positive     0.025000    0.012500  0.012500  0.012500  0.012500  0.012500   

              brand     broke       but       buy  ...  terrible     there  \
sentiment                                          ...                       
negative   0.025641  0.025641  0.012821  0.025641  ...  0.038462  0.012821   
neutral    0.012821  0.012821  0.051282  0.012821  ...  0.025641  0.038462   
positive   0.012500  0.012500  0.025000  0.012500  ...  0.012500  0.012500   

                use   useless      very       who      will      with  \
sentiment                                                               
negative   0.025641  0.025641  0.012821  0.012821  0.025641  0.012821   
neut

독립성 가정을 이용하여 가능도(likelihood)를 구합니다.  
첫번째 문장 예시: $P(love, great, awesome|positive), P(love, great, awesome|negative), P(love, great, awesome|neutral)$

In [73]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np


# Vectorizer로 변환
vectorizer = CountVectorizer()
review_array = vectorizer.fit_transform(df['review']).toarray()

# 단어 리스트 가져오기
words = vectorizer.get_feature_names_out()

# 단어의 빈도 매트릭스 생성
frequency_matrix = pd.DataFrame(review_array, columns=words)

# 원본 데이터프레임과 단어 빈도 매트릭스 결합
frequency_matrix = pd.concat([df['sentiment'], frequency_matrix], axis=1)

# 클래스별 단어 빈도 합산
class_word_frequencies = frequency_matrix.groupby('sentiment').sum()

# 클래스별 단어 빈도 합계 계산
class_word_totals = class_word_frequencies.sum(axis=1)

# 스무딩 파라미터 설정
alpha = 1

# 조건부 확률 계산
conditional_probabilities = (class_word_frequencies + alpha) / (class_word_totals.to_numpy()[:, np.newaxis] + alpha * len(words))

# 특정 문장에 대한 가능도 계산 함수 정의
def calculate_likelihood(sentence):
    # 문장을 단어로 변환
    sentence_words = sentence.lower().split()

    # 존재하는 단어만 필터링 (벡터화된 단어 목록에 없을 경우를 대비)
    sentence_words = [word for word in sentence_words if word in words]

    likelihoods = {}

    for sentiment in ['positive', 'negative', 'neutral']:
        # 단어들의 조건부 확률 곱 계산
        likelihood = 1
        for word in sentence_words:
            likelihood *= conditional_probabilities.loc[sentiment, word]
        likelihoods[sentiment] = likelihood

    return likelihoods


위에서 구한 사전 확률과 가능도를 이용하여 타겟 문장이 positive, negative, neutral일 확률을 구하고 최종적으로 어떤 감성일지 분석해봅니다.

In [77]:
import numpy as np
# 최종 확률 구하는 코드를 작성해주세요.
# 타겟 문장
example_sentence = "love great awesome"

# 사전 확률
prior_probabilities = {
    'positive': 4 / 10,
    'negative': 3 / 10,
    'neutral': 3 / 10
}

# 가능도
likelihoods = calculate_likelihood(example_sentence)

# 확률 계산 (베이즈 정리 적용)
posterior_probabilities = {}

for sentiment in ['positive', 'negative', 'neutral']:
    posterior_probabilities[sentiment] = likelihoods[sentiment] * prior_probabilities[sentiment]

# 정규화
total_posterior = sum(posterior_probabilities.values())
for sentiment in posterior_probabilities:
    posterior_probabilities[sentiment] /= total_posterior

# 결과 출력
print(f"Posterior probabilities for '{example_sentence}':")
for sentiment, probability in posterior_probabilities.items():
    print(f"P({example_sentence} | {sentiment}) = {probability}")

# 최종 감성 결정
predicted_sentiment = max(posterior_probabilities, key=posterior_probabilities.get)
print(f"\nThe predicted sentiment for '{example_sentence}' is: {predicted_sentiment}")



Posterior probabilities for 'love great awesome':
P(love great awesome | positive) = 0.926873857404022
P(love great awesome | negative) = 0.03656307129798902
P(love great awesome | neutral) = 0.03656307129798902

The predicted sentiment for 'love great awesome' is: positive


In [78]:
# 두번째 문장
import numpy as np

# 타겟 문장
example_sentence = "terrible, not, never"

# 사전 확률
prior_probabilities = {
    'positive': 4 / 10,
    'negative': 3 / 10,
    'neutral': 3 / 10
}

# 가능도
likelihoods = calculate_likelihood(example_sentence)

# 확률 계산 (베이즈 정리 적용)
posterior_probabilities = {}

for sentiment in ['positive', 'negative', 'neutral']:
    posterior_probabilities[sentiment] = likelihoods[sentiment] * prior_probabilities[sentiment]

# 정규화
total_posterior = sum(posterior_probabilities.values())
for sentiment in posterior_probabilities:
    posterior_probabilities[sentiment] /= total_posterior

# 결과 출력
print(f"Posterior probabilities for '{example_sentence}':")
for sentiment, probability in posterior_probabilities.items():
    print(f"P({example_sentence} | {sentiment}) = {probability}")

# 최종 감성 결정
predicted_sentiment = max(posterior_probabilities, key=posterior_probabilities.get)
print(f"\nThe predicted sentiment for '{example_sentence}' is: {predicted_sentiment}")


Posterior probabilities for 'terrible, not, never':
P(terrible, not, never | positive) = 0.3023255813953489
P(terrible, not, never | negative) = 0.4651162790697674
P(terrible, not, never | neutral) = 0.2325581395348837

The predicted sentiment for 'terrible, not, never' is: negative


A2-1.   
Target review1의 분류 결과:   positve

Target review2의 분류 결과:   negative

Q2-2. 나이브 베이지안 기반 확률을 구하는 과정에서 어떤 문제점을 발견할 수 있었나요? 그리고 그 문제를 해결하기 위한 방법에 대해 간략하게 조사 및 서술해 주세요. (힌트: Laplace smoothing)

A2-2.
나이브 베이즈 분류기를 사용할 때, 특정 단어가 훈련 데이터의 어떤 클래스에서도 나타나지 않는다면 해당 단어의 조건부 확률이 0이 된다. 이런 경우, 베이즈 정리에서 가능도에 이 확률을 곱하게 되므로, 최종 확률(P(y|x))도 0이 되어버린다. 이는 실제로 중요한 단어가 훈련 데이터에 없어서 발생하는 문제로, 모델이 해당 클래스에 대한 확률을 잘못 평가할 수 있다. 이 문제를 해결하기 위해 라플라스 스무딩(Laplace Smoothing)을 적용할 수 있다. 라플라스 스무딩은 각 단어의 빈도에 작은 상수(보통 1)를 더해줌으로써, 어떤 단어의 조건부 확률이 0이 되는 것을 방지하는 기법이다. 라플라스 스무딩을 적용하면, 이전에 빈도가 0이었던 단어들도 매우 작은 확률을 가지게 되므로, 모델이 이 단어들을 무시하지 않고 사용할 수 있다. 이로 인해 모델의 성능이 향상되며, 특히 훈련 데이터가 적은 경우나 특정 단어가 드물게 나타나는 경우에 유용하다.